係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [1]:
import re

In [2]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

In [3]:
class Morph:
    def __init__(self, line):
        
        #タブとカンマで分割
        cols = separator.split(line)
        
        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.srcs = []   # 係り元文節インデックス番号のリスト
        self.dst  = dst  # 係り先文節インデックス番号
        self.phrase = ''.join([morph.surface for morph in morphs if morph.pos!= '記号']) # 文節

In [5]:
# 係り元を代入し、Chunkリストを文のリストを追加
def append_sentence(chunks, sentences):
    
    # 係り元を代入
    for i, chunk in enumerate(chunks):
        if chunk.dst != -1:
            chunks[chunk.dst].srcs.append(i)
    sentences.append(chunks)
    return sentences, []

In [6]:
%time

morphs = []
chunks = []
sentences = []

with open('./neko.txt.cabocha') as f:
    
    for line in f:
        dependancies = dependancy.match(line)
        
        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))
            
        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []
       
        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))
        
        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences, chunks = append_sentence(chunks, sentences)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [7]:
for si, sentence in enumerate(sentences):
    print('-----', si, '-----')
    for ci, chunk in enumerate(sentence):
        if chunk.dst != -1:
            print('{}:{}\t{}'.format(ci, chunk.phrase, sentence[chunk.dst].phrase))
    
    # 多いので制限
    if si > 5:
        break

----- 0 -----
----- 1 -----
----- 2 -----
0:名前は	無い
1:まだ	無い
----- 3 -----
0:どこで	生れた
1:生れた	かとんと
2:かとんと	つかぬ
3:見当が	つかぬ
----- 4 -----
0:何でも	薄暗い
1:薄暗い	泣いて
2:じめじめ	泣いて
3:した所で	泣いて
4:ニャーニャー	泣いて
5:泣いて	記憶している
6:いた事だけは	記憶している
----- 5 -----
0:吾輩は	見た
1:ここで	始めて
2:始めて	人間という
3:人間という	ものを
4:ものを	見た
----- 6 -----
0:しかも	そうだ
1:あとで	聞くと
2:聞くと	そうだ
3:それは	そうだ
4:書生という	人間中で
5:人間中で	種族であった
6:一番	獰悪な
7:獰悪な	種族であった
8:種族であった	そうだ
